# ⚠️ Important Notice

This notebook (and repository) is deprecated.

For the latest python examples, please refer to the `llama-cloud-services` repository examples: 
https://github.com/run-llama/llama_cloud_services/tree/main/examples

---

# Composite Retrieval - Searching over multiple indexes at once

On this notebook, we'll be running retrieval over two different indexes, creating a composite retrieval which makes easier to manage multiple different indexes.

In [ ]:
%pip install llama-index
%pip install llama-index-indices-managed-llama-cloud

## Setup

In [2]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
import os
# API access to llama-cloud

os.environ["LLAMA_CLOUD_API_KEY"] = ""

In [4]:
# Using OpenAI API for embeddings/llms
os.environ["OPENAI_API_KEY"] = ""

## Load Documents into LlamaCloud
The first order of business is to download the 5 Apple and Tesla 10Ks and upload them into LlamaCloud.

You can easily do this by creating a pipeline and uploading docs via the "Files" mode.

After this is done, proceed to the next section.

## Define llama-cloud indexes

Now we'll be defining the indexes we'll be adding to our new Retriever

In [5]:
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex
import os

project_name = "Default"
organization_id = "359ed263-e37a-4bf4-a2f1-3b9f89c18a86"

apple_index = LlamaCloudIndex(
  name="apple_index",
  project_name=project_name,
  api_key=os.environ["LLAMA_CLOUD_API_KEY"],
  organization_id=organization_id
)

tesla_index = LlamaCloudIndex(
  name="tesla_index",
  project_name="Default",
  api_key=os.environ["LLAMA_CLOUD_API_KEY"],
  organization_id=organization_id
)

## Creating Retriever

A Composite Retriever lets you combine multiple indexes into a single retriever, making it easier to query all of them at once. It is also saved in LlamaCloud, so you can reuse it later without recreating it.

Why use a Composite Retriever?
- Unified Search: Query multiple indexes at once.
- Reusability: Saved in LlamaCloud, so you can use it again later.
- Easy to Use: Simple to configure and perfect for projects with multiple indexes

In [6]:
from llama_index.indices.managed.llama_cloud import LlamaCloudCompositeRetriever
from llama_cloud import CompositeRetrievalMode

composite_retriever = LlamaCloudCompositeRetriever(
    name="Apple/Tesla Retriever",
    project_name=project_name,
    organization_id=organization_id,
    # If a Retriever named "Apple/Tesla Retriever" doesn't already exist, one will be created
    create_if_not_exists=True,
    # CompositeRetrievalMode.FULL will query each index individually and globally rerank results at the end
    # CompositeRetrievalMode.ROUTED an agent determines which sub-indices are most relevant to the provided query (based on the sub-index's name & description you've provided)
    mode=CompositeRetrievalMode.FULL,
    # return the top 5 results from all queried indices
    rerank_top_n=5,
)

Adding indexes

In [7]:
composite_retriever.add_index(
    apple_index,
    description="Information from apple 10K files",
)
composite_retriever.add_index(
    tesla_index,
    description="Information from tesla 10K files",
)

Retriever(name='Apple/Tesla Retriever', pipelines=[RetrieverPipeline(name='apple_index', description='Information from apple 10K files', pipeline_id='430f37e6-2f01-4b53-bd68-0597eef987b2', preset_retrieval_parameters=PresetRetrievalParams(dense_similarity_top_k=30, dense_similarity_cutoff=0.0, sparse_similarity_top_k=30, enable_reranking=True, rerank_top_n=6, alpha=0.5, search_filters=None, files_top_k=1, retrieval_mode=<RetrievalMode.CHUNKS: 'chunks'>, retrieve_image_nodes=False, class_name='base_component')), RetrieverPipeline(name='tesla_index', description='Information from tesla 10K files', pipeline_id='383feb9d-7b70-4256-825b-df5bf2162217', preset_retrieval_parameters=PresetRetrievalParams(dense_similarity_top_k=30, dense_similarity_cutoff=0.0, sparse_similarity_top_k=30, enable_reranking=True, rerank_top_n=6, alpha=0.5, search_filters=None, files_top_k=1, retrieval_mode=<RetrievalMode.CHUNKS: 'chunks'>, retrieve_image_nodes=False, class_name='base_component'))], id='cae87332-eee

In [8]:
nodes = composite_retriever.retrieve("What are the tiny risks factors for apple 2019")

In [9]:
print(nodes)

[NodeWithScore(node=TextNode(id_='5a439349-a87e-491b-a237-27bdae69b7ec', embedding=None, metadata={'file_size': 855181, 'last_modified_at': '2025-01-29T21:15:29', 'file_path': 'apple_2019.pdf', 'file_name': 'apple_2019.pdf', 'external_file_id': 'apple_2019.pdf', 'file_id': '06e59e59-5375-4681-a17c-03a684e6175d', 'pipeline_file_id': 'd8239b91-d688-4528-8de0-9af84799deab', 'pipeline_id': '430f37e6-2f01-4b53-bd68-0597eef987b2', 'page_label': 11, 'start_page_index': 10, 'start_page_label': 11, 'end_page_index': 10, 'end_page_label': 11, 'document_id': '4004d1ff9e7fcd8fd6a06f5279dee81ed5a4ae7f8add7f9b74', 'start_char_idx': 42666, 'end_char_idx': 47959, 'retriever_id': 'cae87332-eee5-40dc-a81e-36d53643ab01', 'retriever_pipeline_name': 'apple_index'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='## Item 1A. Risk Factors\n\nThe following discussion of risk factors contains forward-looking st

## OpenAI Agent Over Composite Retriever

In [10]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o")

query_engine = RetrieverQueryEngine.from_args(
    composite_retriever,
    llm=llm
)

In [11]:
from llama_index.core.tools import FunctionTool, ToolMetadata, QueryEngineTool

tool_appletesla = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="composed_apple_tesla_retriever",
        # specific prompt to
        description="Useful for information about apple and tesla 10ks"
    ),
)

In [12]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

llm_agent = OpenAI(model="gpt-4o", system_prompt="You're an agent with access to knowledge base")

agent = FunctionCallingAgentWorker.from_tools(
    [tool_appletesla], llm=llm_agent, verbose=True
).as_agent()

In [13]:
response = agent.chat("What's is the Revenue from Apple in 2020")

print(str(response))

Added user message to memory: What's is the Revenue from Apple in 2020
=== Calling Function ===
Calling function: composed_apple_tesla_retriever with args: {"input": "Apple revenue 2020"}
=== Function Output ===
In 2020, Apple's total net sales amounted to $274,515 million.
=== LLM Response ===
In 2020, Apple's total revenue was $274,515 million.
In 2020, Apple's total revenue was $274,515 million.


In [14]:
response = agent.chat("What's is the Revenue from Tesla in 2020")


Added user message to memory: What's is the Revenue from Tesla in 2020
=== Calling Function ===
Calling function: composed_apple_tesla_retriever with args: {"input": "Tesla revenue 2020"}
=== Function Output ===
In 2020, Tesla recognized total revenues of $31.54 billion.
=== LLM Response ===
In 2020, Tesla's total revenue was $31.54 billion.
